<a href="https://colab.research.google.com/github/shahulrashid/internship_tasks/blob/main/Face_Mask_Detection_with_Live_Alert_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split



In [2]:
data = []
labels = []
image_size = 100

for category, label in [('with_mask', 0), ('without_mask', 1)]:
    folder = f"dataset/{category}"
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (image_size, image_size))
            data.append(img)
            labels.append(label)

data = np.array(data) / 255.0
labels = to_categorical(labels)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)


FileNotFoundError: [Errno 2] No such file or directory: 'dataset/with_mask'

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout



In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(100,100,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
model.save("mask_detector.h5")


In [3]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np


In [4]:
model = load_model("mask_detector.h5")
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
labels_dict = {0: 'Mask', 1: 'No Mask'}
color_dict = {0: (0,255,0), 1: (0,0,255)}

cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for x,y,w,h in faces:
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (100,100))
        face = np.expand_dims(face, axis=0)/255.0
        pred = model.predict(face)[0]
        label = np.argmax(pred)

        cv2.rectangle(frame, (x,y), (x+w,y+h), color_dict[label], 2)
        cv2.putText(frame, labels_dict[label], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color_dict[label], 2)

        # Trigger alert
        if label == 1:
            print("⚠️ ALERT: No Mask Detected!")

    cv2.imshow("Live Face Mask Detector", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'mask_detector.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
from flask import Flask, render_template, Response
import cv2

app = Flask(__name__)
camera = cv2.VideoCapture(0)

def generate_frames():
    while True:
        success, frame = camera.read()
        if not success:
            break
        else:
            # Add face + mask detection logic here
            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/')
def index():
    return render_template('index.html')  # simple HTML page to stream

@app.route('/video')
def video():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
